In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.font_manager as font_manager
from matplotlib import rc
import f90nml

rc('text', usetex=True)
rc('font', family='serif')
thin = 1
fs = 14

def read_control(experiment):
    namelist = f90nml.read(
        '../data/' + experiment + '/' + experiment + '.out')
    tmp = namelist['nml_parameter']
    N_log_scale = tmp['N_log_scale']
    M_log_scale = tmp['M_log_scale']
    N_time = tmp['N_time']
    NN = tmp['NN']
    NM = tmp['NM']
    NR = tmp['NR']
    NE = tmp['NE']
    N_min = tmp['N_min']
    N_max = tmp['N_max']
    M_min = tmp['M_min']
    M_max = tmp['M_max']
    Ro_min = tmp['Ro_min']
    Ro_max = tmp['Ro_max']
    e_min = tmp['e_min']
    e_max = tmp['e_max']
    figure_directory = "../figure/" + experiment
    if not os.path.isdir(figure_directory):
        os.makedirs(figure_directory)
    return (N_log_scale, M_log_scale, N_time, NN, NM, NR, NE, N_min, N_max,
            M_min, M_max, Ro_min, Ro_max, e_min, e_max, figure_directory)

In [ ]:
IN = 1
c_levels = np.linspace(0, 0.06, 30)
thin = 10

fig = plt.figure(figsize=[8, 3])
fig.subplots_adjust(left=0.09, bottom=0.15, right=0.89,
                    top=0.92, wspace=0.25, hspace=0.2)

experiment = 'floquet_NM10000_NE10'
(N_log_scale, M_log_scale, N_time, NN, NM, NR, NE, N_min, N_max, M_min, M_max,
 Ro_min, Ro_max,e_min, e_max, figure_directory) = read_control(experiment)

log_M_axis = np.linspace(np.log(M_min), np.log(M_max), NM, endpoint=True)
M_axis = np.exp(log_M_axis)
Ro_axis = np.linspace(Ro_min, Ro_max, NR, endpoint=True)
e_axis = np.linspace(e_min, e_max, NE, endpoint=True)

e = 0.6
ie = np.where(np.isclose(e_axis, e))[0][0]
shape = np.array([NR, NM])
size = shape.prod()
data_type = np.dtype('<f8')
data_bytes = round(int(data_type.name[-2:]) / 8)
offset = size * data_bytes * ie
file_open = open(
    '../data/' + experiment + '/growth_rate_E{:0>6}.out'.format(IN), 'r')
data_tmp = np.fromfile(file_open, dtype=data_type, count=size,
                       offset=offset)
growth_rate_Ro = data_tmp.reshape(shape, order='F')
growth_rate_Ro[np.isclose(growth_rate_Ro, 0, atol=1.e-4)] = 0

ax1 = fig.add_subplot(121)
cont1 = ax1.contourf(Ro_axis, M_axis[::thin], 
                     growth_rate_Ro[:, ::thin].transpose(),
                     levels=c_levels, cmap=cm.jet, extend="max")
ax1.scatter([0.95, 0.95, 0.95], [57.21, 27.94, 18.50],
            marker='D', color='w', edgecolor='k')
ax1.set_yscale('log')

grid_x_ticks_minor = np.arange(0.5, 1.01, 0.01)
grid_x_ticks_major = np.arange(0.5, 1.1, 0.1)
ax1.set_xticks(grid_x_ticks_minor, minor=True)
ax1.set_xticks(grid_x_ticks_major, major=True)
ax1.set_ylim(5.e0, 5.e2)
ax1.set_xlim(0.5, 1.0)

ax1.set_ylabel(r'$\tan \phi$', fontsize=fs)
ax1.set_xlabel(r'$Ro$', fontsize=fs)
ax1.set_title('(a)', fontsize=fs, loc='left')

experiment = 'floquet_NM10000_NR11'
(N_log_scale, M_log_scale, N_time, NN, NM, NR, NE, N_min, N_max, M_min, M_max,
 Ro_min, Ro_max,e_min, e_max, figure_directory) = read_control(experiment)

log_M_axis = np.linspace(np.log(M_min), np.log(M_max), NM, endpoint=True)
M_axis = np.exp(log_M_axis)
Ro_axis = np.linspace(Ro_min, Ro_max, NR, endpoint=True)
e_axis = np.linspace(e_min, e_max, NE, endpoint=True)

Ro = 0.95
ir = np.where(np.isclose(Ro_axis, Ro))[0][0]
shape = np.array([NE, NM])
size = shape.prod()
data_type = np.dtype('<f8')
data_bytes = round(int(data_type.name[-2:]) / 8)
offset = size * data_bytes * ir
print(ir, size, data_bytes, offset)
file_open = open(
    '../data/' + experiment + '/growth_rate_R{:0>6}.out'.format(IN), 'r')
data_tmp = np.fromfile(file_open, dtype=data_type, count=size,
                       offset=offset)
growth_rate_e = data_tmp.reshape(shape, order='F')
growth_rate_e[np.isclose(growth_rate_e, 0, atol=1.e-4)] = 0

ax2 = fig.add_subplot(122)
cont2 = ax2.contourf(e_axis, M_axis[::thin], 
                     growth_rate_e[:, ::thin].transpose(),
                     levels=c_levels, cmap=cm.jet, extend="max")
ax2.scatter([0.6, 0.6, 0.6], [57.21, 27.94, 18.50],
            marker='D', color='w', edgecolor='k')
ax2.set_yscale('log')

grid_x_ticks_minor = np.arange(0.0, 1.1, 0.1)
grid_x_ticks_major = np.arange(0.0, 1.2, 0.2)
ax2.set_xticks(grid_x_ticks_minor, minor=True)
ax2.set_xticks(grid_x_ticks_major, major=True)
ax2.set_ylim(5.e0, 5.e2)
ax2.set_xlim(0.0, 1.0)
ax2.set_ylabel(r'$\tan \phi$', fontsize=fs)
ax2.set_xlabel(r'$e$', fontsize=fs)
ax2.set_title('(b)', fontsize=fs, loc='left')
cax = fig.add_axes((0.91, 0.155, 0.012, 0.76))
cb = plt.colorbar(cont2, ticks=[0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06],
                  orientation='vertical', cax=cax,
                  label=r"$\lambda/f$")
text = cax.yaxis.label
font = font_manager.FontProperties(size=fs)
text.set_font_properties(font)

figure_name = figure_directory + '/' + 'growth_rate_Ro_e'
fig.savefig(figure_name + '.eps')
fig.savefig(figure_name + '.png')

In [ ]:
experiment = 'floquet'
(N_log_scale, M_log_scale, N_time, NN, NM, NR, NE, N_min, N_max, M_min, M_max,
 Ro_min, Ro_max,e_min, e_max, figure_directory) = read_control(experiment)

log_M_axis = np.linspace(np.log(M_min), np.log(M_max), NM, endpoint=True)
M_axis = np.exp(log_M_axis)
Ro_axis = np.linspace(Ro_min, Ro_max, NR, endpoint=True)
e_axis = np.linspace(e_min, e_max, NE, endpoint=True)

IN = 1
shape = np.array([NR, NE])
size = shape.prod()
data_type = np.dtype('<f8')
data_bytes = round(int(data_type.name[-2:]) / 8)
file_open = open(
    '../data/floquet/growth_rate_max{:0>6}.out'.format(IN), 'r')
data_tmp = np.fromfile(file_open, dtype=data_type, count=size)
growth_rate_max = data_tmp.reshape(shape, order='F')

colors = {0.1:'r', 0.2:'m', 0.4:'c', 0.6:'g', 0.8:'b', 0.9:'purple'}
def plot_growth_against_1Ro(e, ax):
    ie = np.where(np.isclose(e_axis, e))[0][0]
    growth_rate_Ro = growth_rate_max[:, ie]
    ax.plot(1.0/Ro_axis, growth_rate_Ro, c=colors[e],
               label=r'$e = {:.1g}$'.format(e), zorder=20)

fig = plt.figure(figsize=[8, 3])
fig.subplots_adjust(left=0.09, bottom=0.15, right=0.89,
                    top=0.92, wspace=0.24, hspace=0.2)
ax1 = fig.add_subplot(121)
plot_growth_against_1Ro(0.1, ax1)
plot_growth_against_1Ro(0.2, ax1)
plot_growth_against_1Ro(0.4, ax1)
plot_growth_against_1Ro(0.6, ax1)
plot_growth_against_1Ro(0.8, ax1)
plot_growth_against_1Ro(0.9, ax1)
ax1.legend()


grid_x_ticks_minor = np.arange(1.0, 2.1, 0.1)
grid_x_ticks_major = np.arange(1.0, 2.2, 0.2)
ax1.set_xticks(grid_x_ticks_minor, minor=True)
ax1.set_xticks(grid_x_ticks_major, major=True)
ax1.set_ylim(1.e-4, 1.e-1)
ax1.set_xlim(1.0, 2.0)

ax1.grid(zorder=10)
ax1.grid(which='minor')
ax1.set_xlabel(r'$1/Ro$', fontsize=fs)
ax1.set_ylabel(r'$\lambda_{\rm max}/f$', fontsize=fs)
ax1.set_yscale('log');
ax1.set_title('(a)', fontsize=fs, loc='left')

Ro, e = np.meshgrid(Ro_axis, e_axis)
A_minus_S = 1 - Ro - (1 - (1 - e)**2)/(1 + (1 - e)**2) * Ro
c_levels = np.linspace(0, 0.06, 100)

ax2 = fig.add_subplot(122)
cont = ax2.contourf(Ro_axis, e_axis, growth_rate_max.transpose(),
                    levels=c_levels, cmap=cm.jet, extend="max")
ax2.contour(Ro, e, A_minus_S, [0], colors=['w'])

grid_x_ticks_minor = np.arange(0.5, 1.01, 0.01)
grid_x_ticks_major = np.arange(0.5, 1.1, 0.1)
grid_y_ticks_minor = np.arange(0.0, 1.1, 0.1)
grid_y_ticks_major = np.arange(0.0, 1.2, 0.2)
ax2.set_xticks(grid_x_ticks_minor, minor=True)
ax2.set_xticks(grid_x_ticks_major, major=True)
ax2.set_yticks(grid_y_ticks_minor, minor=True)
ax2.set_yticks(grid_y_ticks_major, major=True)
ax2.set_xlim(0.5, 1.0)
ax2.set_ylim(0.0, 1.0)
ax2.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
ax2.set_ylabel(r'$e$', fontsize=fs)
ax2.set_xlabel(r'$Ro$', fontsize=fs)
ax2.set_title('(b)', fontsize=fs, loc='left')
cax = fig.add_axes((0.91, 0.16, 0.012, 0.75))
fig.colorbar(cont, ticks=[0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06],
             orientation='vertical', cax=cax, ax=ax2,
             label=r"$\lambda_{\rm max}/f$");
text = cax.yaxis.label
font = font_manager.FontProperties(size=fs)
text.set_font_properties(font)

figure_name = figure_directory + '/' +  'growth_rate'
fig.savefig(figure_name + '.eps')
fig.savefig(figure_name + '.png')
print(NM)